*DOMINO-SEE构建网络的效率测试（ECA）*

测试基准：
+ 时空分辨率：1度分辨率（360x180），30年日数据（10957天）
+ 网络构建方法：基于ECA的显著 precursor & trigger rates 
+ 计算能力：24核双线程 2.4GHz

In [ ]:
import xarray as xr
from dask.distributed import Client, performance_report
client = Client(processes=False)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.245.92.223:8787/status,
Dashboard: http://10.245.92.223:8787/status,Workers: 1
Total threads: 48,Total memory: 251.39 GiB
Status: running,Using processes: False
Comm: inproc://10.245.92.223/1592091/9,Workers: 1
Dashboard: http://10.245.92.223:8787/status,Total threads: 48
Started: Just now,Total memory: 251.39 GiB
Comm: inproc://10.245.92.223/1592091/12,Total threads: 48
Dashboard: http://10.245.92.223:34879/status,Memory: 251.39 GiB
Nanny: None,


In [28]:
ds = xr.open_dataset("tests/data/era5.reanalysis.spi30d.0p25deg.china.1950-1979.nc").rename({"latitude": "lat", "longitude": "lon", "spi30d": "SPI1"})\
        .chunk({"lon": 20, "lat": 20})
ds.SPI1

<xarray.DataArray 'SPI1' (time: 10957, lat: 203, lon: 250)> Size: 2GB
dask.array<xarray-SPI1, shape=(10957, 203, 250), dtype=float32, chunksize=(10957, 20, 20), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 88kB 1950-01-01 1950-01-02 ... 1979-12-31
  * lon      (lon) float64 2kB 73.0 73.25 73.5 73.75 ... 134.5 134.8 135.0 135.2
  * lat      (lat) float64 2kB 54.25 54.0 53.75 53.5 53.25 ... 4.5 4.25 4.0 3.75
Attributes:
    units:               1
    calibration_period:  
    freq:                D
    window:              30
    scipy_dist:          gamma
    method:              PWM
    group:               time.dayofyear
    time_indexer:        {}

# Get events

In [6]:
from dominosee.eventorize import get_events
da_drought = get_events(ds.SPI1, threshold=-1.0, extreme="below", outname="drought").compute().chunk({"lat": 20, "lon": 20})
da_drought

<xarray.DataArray 'drought' (time: 10957, lat: 100, lon: 100)> Size: 110MB
dask.array<xarray-<this-array>, shape=(10957, 100, 100), dtype=bool, chunksize=(10957, 20, 20), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 88kB 1950-01-01 1950-01-02 ... 1979-12-31
  * lon      (lon) float64 800B 73.0 73.25 73.5 73.75 ... 97.0 97.25 97.5 97.75
  * lat      (lat) float64 800B 54.25 54.0 53.75 53.5 ... 30.25 30.0 29.75 29.5
Attributes:
    threshold:    -1.0
    extreme:      below
    long_name:    drought events
    description:  Events with -1.0 below threshold

# Get windows

In [7]:
from dominosee.eca import get_eca_precursor_window, get_eca_trigger_window
da_drought_precwindow = get_eca_precursor_window(da_drought, delt=2, sym=False, tau=0).compute().chunk({"lat": 20, "lon": 20})
# da_drought_precwindow
da_drought_trigwindow = get_eca_trigger_window(da_drought, delt=2, sym=False, tau=0).compute().chunk({"lat": 20, "lon": 20})
da_drought_trigwindow

/home/hmwang-cli/.conda/envs/syncfuture/lib/python3.11/site-packages/distributed/client.py:3245: UserWarning: Sending large graph of size 104.50 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/hmwang-cli/.conda/envs/syncfuture/lib/python3.11/site-packages/distributed/client.py:3245: UserWarning: Sending large graph of size 104.50 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


<xarray.DataArray 'drought' (lat: 100, lon: 100, time: 10957)> Size: 110MB
dask.array<xarray-<this-array>, shape=(100, 100, 10957), dtype=bool, chunksize=(20, 20, 10957), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 88kB 1950-01-01 1950-01-02 ... 1979-12-31
  * lon      (lon) float64 800B 73.0 73.25 73.5 73.75 ... 97.0 97.25 97.5 97.75
  * lat      (lat) float64 800B 54.25 54.0 53.75 53.5 ... 30.25 30.0 29.75 29.5
Attributes:
    long_name:    Trigger Window
    units:        1
    description:  Window for trigger event identification
    eca_params:   {"delt": 2, "sym": false, "tau": 0}

# Calculate eca and its confidence

In [8]:
from dominosee.eca import get_eca_precursor

In [9]:
da_drought_prec = get_eca_precursor(da_drought_precwindow, da_drought, func="guvectorize").compute()
da_drought_prec

/home/hmwang-cli/.conda/envs/syncfuture/lib/python3.11/site-packages/dask/array/gufunc.py:458: PerformanceWarning: Increasing number of chunks by factor of 25
  tmp = blockwise(
/home/hmwang-cli/.conda/envs/syncfuture/lib/python3.11/site-packages/distributed/client.py:3245: UserWarning: Sending large graph of size 209.10 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


<xarray.DataArray 'drought' (latA: 100, lonA: 100, latB: 100, lonB: 100)> Size: 200MB
array([[[[1822, 1761, 1696, ...,  470,  430,  436],
         [1717, 1687, 1655, ...,  500,  437,  424],
         [1629, 1612, 1622, ...,  599,  549,  481],
         ...,
         [ 371,  389,  408, ...,  348,  294,  270],
         [ 356,  391,  379, ...,  322,  327,  272],
         [ 345,  352,  358, ...,  323,  362,  294]],

        [[1762, 1808, 1737, ...,  482,  443,  446],
         [1687, 1711, 1689, ...,  506,  444,  433],
         [1623, 1625, 1646, ...,  604,  551,  480],
         ...,
         [ 356,  373,  392, ...,  347,  286,  259],
         [ 340,  376,  367, ...,  310,  321,  267],
         [ 338,  345,  350, ...,  307,  353,  287]],

        [[1688, 1731, 1805, ...,  496,  458,  460],
         [1657, 1696, 1710, ...,  526,  470,  455],
         [1620, 1625, 1644, ...,  622,  572,  505],
         ...,
...
         ...,
         [ 280,  275,  265, ..., 1156, 1146, 1153],
         [ 279,  305,  298, ..., 1542, 1244, 1194],
         [ 260,  273,  289, ..., 1878, 1458, 1283]],

        [[ 382,  371,  368, ...,  353,  346,  360],
         [ 357,  354,  355, ...,  306,  322,  347],
         [ 380,  372,  367, ...,  309,  340,  351],
         ...,
         [ 226,  222,  239, ..., 1430, 1375, 1303],
         [ 217,  243,  257, ..., 1456, 1640, 1398],
         [ 209,  226,  244, ..., 1467, 1919, 1508]],

        [[ 319,  309,  298, ...,  308,  303,  298],
         [ 290,  290,  289, ...,  243,  268,  296],
         [ 306,  305,  296, ...,  250,  277,  283],
         ...,
         [ 203,  204,  218, ..., 1283, 1304, 1287],
         [ 202,  231,  251, ..., 1312, 1443, 1465],
         [ 191,  217,  236, ..., 1298, 1564, 1664]]]], dtype=uint16)
Coordinates:
  * lonA     (lonA) float64 800B 73.0 73.25 73.5 73.75 ... 97.0 97.25 97.5 97.75
  * latA     (latA) float64 800B 54.25 54.0 53.75 53.5 ... 30.25 30.0 29.75 29.5
  * lonB     (lonB) float64 800B 73.0 73.25 73.5 73.75 ... 97.0 97.25 97.5 97.75
  * latB     (latB) float64 800B 54.25 54.0 53.75 53.5 ... 30.25 30.0 29.75 29.5
Attributes:
    long_name:    Precursor Events
    units:        count
    description:  Number of precursor events (from location A to location B) ...
    eca_params:   {"delt": 2, "sym": false, "tau": 0}

In [10]:
from dominosee.eca import get_eca_precursor_confidence

In [11]:
da_drought_prec_conf = get_eca_precursor_confidence(da_drought_prec.chunk(20), da_drought.chunk({"lat": 20, "lon": 20}), da_drought.chunk({"lat": 20, "lon": 20})).compute()
da_drought_prec_conf

/home/hmwang-cli/.conda/envs/syncfuture/lib/python3.11/site-packages/distributed/client.py:3245: UserWarning: Sending large graph of size 295.41 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


<xarray.DataArray 'prec_conf' (latA: 100, lonA: 100, latB: 100, lonB: 100)> Size: 400MB
array([[[[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
          1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
          1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
          1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         ...,
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
          9.99999881e-01, 9.81336594e-01, 2.82211721e-01],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
          9.99870718e-01, 9.98942494e-01, 8.63806427e-01],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
          9.88180280e-01, 9.99987960e-01, 9.95944798e-01]],

        [[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
          1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
          1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
          1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
...
          1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [6.96050189e-03, 6.83121085e-02, 3.34030390e-01, ...,
          1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [4.02635988e-03, 6.75759017e-02, 5.02192676e-01, ...,
          1.00000000e+00, 1.00000000e+00, 1.00000000e+00]],

        [[9.99979734e-01, 9.99845207e-01, 9.98298705e-01, ...,
          9.82228756e-01, 9.86158073e-01, 9.66803670e-01],
         [9.77242053e-01, 9.88456130e-01, 9.83766019e-01, ...,
          6.44521415e-02, 7.14622974e-01, 9.86905038e-01],
         [9.96625602e-01, 9.98629332e-01, 9.88446534e-01, ...,
          8.18490610e-02, 4.61159378e-01, 8.71723711e-01],
         ...,
         [1.30332774e-02, 4.51051518e-02, 1.04937725e-01, ...,
          1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [8.14200565e-02, 4.63563740e-01, 9.20703351e-01, ...,
          1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [3.14401314e-02, 5.06030381e-01, 9.46361721e-01, ...,
          1.00000000e+00, 1.00000000e+00, 1.00000000e+00]]]],
      dtype=float32)
Coordinates:
  * lonA     (lonA) float64 800B 73.0 73.25 73.5 73.75 ... 97.0 97.25 97.5 97.75
  * latA     (latA) float64 800B 54.25 54.0 53.75 53.5 ... 30.25 30.0 29.75 29.5
  * lonB     (lonB) float64 800B 73.0 73.25 73.5 73.75 ... 97.0 97.25 97.5 97.75
  * latB     (latB) float64 800B 54.25 54.0 53.75 53.5 ... 30.25 30.0 29.75 29.5
Attributes:
    long_name:    Precursor confidence
    units:        
    description:  Confidence of precursor rates (from location A to location ...
    eca_params:   {"delt": 2, "sym": false, "tau": 0}

# Get links

In [12]:
from dominosee.network import get_link_from_confidence

In [13]:
da_link = get_link_from_confidence(da_drought_prec_conf, 0.99)
da_link

<xarray.DataArray 'prec_conf' (latA: 100, lonA: 100, latB: 100, lonB: 100)> Size: 100MB
array([[[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True, False, False],
         [ True,  True,  True, ...,  True,  True, False],
         [ True,  True,  True, ..., False,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True, False, False],
         [ True,  True,  True, ...,  True,  True, False],
         [ True,  True,  True, ..., False,  True, False]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
...
         ...,
         [False, False, False, ...,  True,  True,  True],
         [False,  True,  True, ...,  True,  True,  True],
         [False, False,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ..., False, False,  True],
         [ True,  True,  True, ..., False, False,  True],
         ...,
         [False, False, False, ...,  True,  True,  True],
         [False, False, False, ...,  True,  True,  True],
         [False, False, False, ...,  True,  True,  True]],

        [[ True,  True,  True, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [ True,  True, False, ..., False, False, False],
         ...,
         [False, False, False, ...,  True,  True,  True],
         [False, False, False, ...,  True,  True,  True],
         [False, False, False, ...,  True,  True,  True]]]])
Coordinates:
  * lonA     (lonA) float64 800B 73.0 73.25 73.5 73.75 ... 97.0 97.25 97.5 97.75
  * latA     (latA) float64 800B 54.25 54.0 53.75 53.5 ... 30.25 30.0 29.75 29.5
  * lonB     (lonB) float64 800B 73.0 73.25 73.5 73.75 ... 97.0 97.25 97.5 97.75
  * latB     (latB) float64 800B 54.25 54.0 53.75 53.5 ... 30.25 30.0 29.75 29.5

# Generate ECA in one step calculation

In [20]:
from dominosee.eventorize import get_events
from dominosee.eca import get_eca_precursor_from_events, get_eca_trigger_from_events, get_eca_precursor_confidence, get_eca_trigger_confidence
from dominosee.network import get_link_from_confidence

In [21]:
ds = xr.open_dataset("tests/data/era5.reanalysis.spi30d.0p25deg.china.1950-1979.nc").rename({"latitude": "lat", "longitude": "lon"})\
        .isel(lat=slice(0, 100), lon=slice(0, 100)).chunk({"lat": 50, "lon": 50})
da_drought = get_events(ds.spi30d, threshold=-1.0, extreme="below", outname="drought")
da_drought

<xarray.DataArray 'drought' (time: 10957, lat: 100, lon: 100)> Size: 110MB
dask.array<le, shape=(10957, 100, 100), dtype=bool, chunksize=(10957, 50, 50), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 88kB 1950-01-01 1950-01-02 ... 1979-12-31
  * lon      (lon) float64 800B 73.0 73.25 73.5 73.75 ... 97.0 97.25 97.5 97.75
  * lat      (lat) float64 800B 54.25 54.0 53.75 53.5 ... 30.25 30.0 29.75 29.5
Attributes:
    threshold:    -1.0
    extreme:      below
    long_name:    drought events
    description:  Events with -1.0 below threshold

In [22]:
da_prec = get_eca_precursor_from_events(da_drought, da_drought, delt=2, sym=False, tau=0, func="guvectorize")
da_trig = get_eca_trigger_from_events(da_drought, da_drought, delt=2, sym=False, tau=0, func="guvectorize")

In [23]:
da_prec_conf = get_eca_precursor_confidence(da_prec, da_drought, da_drought)
da_trig_conf = get_eca_trigger_confidence(da_trig, da_drought, da_drought)

In [24]:
da_link = get_link_from_confidence(da_prec_conf, 0.99) & get_link_from_confidence(da_trig_conf, 0.99)
da_link

<xarray.DataArray (latA: 100, lonA: 100, latB: 100, lonB: 100)> Size: 100MB
dask.array<and_, shape=(100, 100, 100, 100), dtype=bool, chunksize=(50, 50, 50, 50), chunktype=numpy.ndarray>
Coordinates:
  * lonA     (lonA) float64 800B 73.0 73.25 73.5 73.75 ... 97.0 97.25 97.5 97.75
  * latA     (latA) float64 800B 54.25 54.0 53.75 53.5 ... 30.25 30.0 29.75 29.5
  * lonB     (lonB) float64 800B 73.0 73.25 73.5 73.75 ... 97.0 97.25 97.5 97.75
  * latB     (latB) float64 800B 54.25 54.0 53.75 53.5 ... 30.25 30.0 29.75 29.5

In [25]:
%%time
da_link_compute = da_link.compute()

CPU times: user 44min 56s, sys: 45.6 s, total: 45min 42s
Wall time: 3min 17s


In [26]:
client.close()